In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import LinearIVGMM
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta

data_cinf = pd.read_parquet("data_cinf.parquet")
data_cpi = pd.read_parquet("data_inf.parquet")
# Filter data from 3Q 2004 onwards
start_date = pd.to_datetime('2004-07-01')
data_cinf = data_cinf[start_date:]
data_cpi = data_cpi[start_date:]

In [2]:
# Sensitivity analysis using actual inflation at time t

# Get CPI index data
data_cpi = pd.read_parquet("data_inf.parquet")
# Define the date for the third quarter of 2004
cutoff_date = datetime(2004, 7, 1)
# Filter the DataFrame to keep data after the cutoff date
data_cpi = data_cpi[data_cpi.index >= cutoff_date]

# OLS
data_cpi1 = data_cpi.dropna(subset=['output_gap_plucking'])
X = sm.add_constant(data_cpi1[['inf_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cpi1['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_inf_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_plucking']]

data_cpi2 = data_cpi.dropna(subset=['output_gap_bb_avg'])
X = sm.add_constant(data_cpi2[['inf_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cpi2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_inf_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_bb_avg']]


data_cpi3 = data_cpi.dropna(subset=['output_gap_rgdp_yoy'])
X = sm.add_constant(data_cpi3[['inf_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cpi3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['inf_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_inf_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_inf_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_rgdp_yoy']]

data_cpi4 = data_cpi.dropna(subset=['output_gap_rbb'])
X = sm.add_constant(data_cpi4[['inf_dev', 'output_gap_rbb', 'opr_lag']])
model = sm.OLS(data_cpi4['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_dev']
coef_output_gap = coefficients['output_gap_rbb']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_rbb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_rbb'] / (1 - alpha)]
results_table_ols_inf_rbb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_rbb']]


# GMM
exog = np.column_stack((np.ones_like(data_cpi1['opr_lag']), data_cpi1['inf_dev'], data_cpi1['output_gap_plucking'], data_cpi1['opr_lag']))
endog = data_cpi1['opr']
instruments = np.roll(endog, 1)[2:]
instruments_1 = np.roll(data_cpi1['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cpi1['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]
instruments_4 = np.roll(data_cpi1['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cpi1['inf'], 2)[2:]
constant = np.ones_like(endog)[2:] 
endog = endog[2:] 
exog = exog[2:] 
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant)) 
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_inf_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi2['opr_lag']), data_cpi2['inf_dev'], data_cpi2['output_gap_bb_avg'], data_cpi2['opr_lag']))
endog = data_cpi2['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cpi2['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi2['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cpi2['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi2['inf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_inf_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi3['opr_lag']), data_cpi3['inf_dev'], data_cpi3['output_gap_rgdp_yoy'], data_cpi3['opr_lag']))
endog = data_cpi3['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cpi3['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cpi3['inf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_inf_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi4['opr_lag']), data_cpi4['inf_dev'], data_cpi4['output_gap_rbb'], data_cpi4['opr_lag']))
endog = data_cpi4['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cpi4['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi4['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cpi4['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi4['inf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rbb = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rbb / (1 - alpha)
results_table_gmm_inf_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_rbb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_rbb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

In [3]:
# Sensitivity analysis using actual inflation at time t

# Get core CPI index data
data_cinf = pd.read_parquet("data_cinf.parquet")
# Define the date for the third quarter of 2004
cutoff_date = datetime(2004, 7, 1)
# Filter the DataFrame to keep data after the cutoff date
data_cinf = data_cinf[data_cinf.index >= cutoff_date]

# OLS
X = sm.add_constant(data_cinf[['cinf_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cinf['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['cinf_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_cinf_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_cinf_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_cinf_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_cinf_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['cinf_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_cinf_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['cinf_dev'], results.pvalues['output_gap_plucking']]

X = sm.add_constant(data_cinf[['cinf_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cinf['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['cinf_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_cinf_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_cinf_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_cinf_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_cinf_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['cinf_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_cinf_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['cinf_dev'], results.pvalues['output_gap_bb_avg']]


data_cinf3 = data_cinf.dropna(subset=['output_gap_rgdp_yoy'])
X = sm.add_constant(data_cinf3[['cinf_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cinf3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['cinf_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_cinf_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_cinf_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_cinf_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_cinf_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['cinf_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_cinf_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['cinf_dev'], results.pvalues['output_gap_rgdp_yoy']]

data_cinf2 = data_cinf.dropna(subset=['output_gap_rbb'])
X = sm.add_constant(data_cinf2[['cinf_dev', 'output_gap_rbb', 'opr_lag']])
model = sm.OLS(data_cinf2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['cinf_dev']
coef_output_gap = coefficients['output_gap_rbb']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_cinf_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_cinf_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_cinf_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_cinf_rbb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['cinf_dev'] / (1 - alpha), results.bse['output_gap_rbb'] / (1 - alpha)]
results_table_ols_cinf_rbb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['cinf_dev'], results.pvalues['output_gap_rbb']]

# GMM
exog = np.column_stack((np.ones_like(data_cinf['opr_lag']), data_cinf['cinf_dev'], data_cinf['output_gap_plucking'], data_cinf['opr_lag']))
endog = data_cinf['opr']
instruments = np.roll(endog, 1)[2:]
instruments_1 = np.roll(data_cinf['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cinf['cinf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]
instruments_4 = np.roll(data_cinf['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cinf['cinf'], 2)[2:]
constant = np.ones_like(endog)[2:] 
endog = endog[2:] 
exog = exog[2:] 
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant)) 
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_cinf_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_cinf_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_cinf_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_cinf_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_cinf_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cinf['opr_lag']), data_cinf['cinf_dev'], data_cinf['output_gap_bb_avg'], data_cinf['opr_lag']))
endog = data_cinf['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cinf['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cinf['cinf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cinf['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cinf['cinf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_cinf_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_cinf_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_cinf_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_cinf_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_cinf_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cinf3['opr_lag']), data_cinf3['cinf_dev'], data_cinf3['output_gap_rgdp_yoy'], data_cinf3['opr_lag']))
endog = data_cinf3['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cinf3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cinf3['cinf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cinf3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cinf3['cinf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_cinf_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_cinf_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_cinf_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_cinf_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_cinf_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cinf2['opr_lag']), data_cinf2['cinf_dev'], data_cinf2['output_gap_rbb'], data_cinf2['opr_lag']))
endog = data_cinf2['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cinf2['output_gap_rbb'], 1)[2:]
instruments_2 = np.roll(data_cinf2['cinf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cinf2['output_gap_rbb'], 2)[2:]
instruments_5 = np.roll(data_cinf2['cinf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rbb = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rbb / (1 - alpha)
results_table_gmm_cinf_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_cinf_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_cinf_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_cinf_rbb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_cinf_rbb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]


In [4]:
# Sensitivity analysis using inf3ma

# OLS
X = sm.add_constant(data_cpi1[['inf_3ma_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cpi1['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_3ma_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf3ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_inf3ma_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_plucking']]

X = sm.add_constant(data_cpi2[['inf_3ma_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cpi2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_3ma_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf3ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_inf3ma_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_bb_avg']]

X = sm.add_constant(data_cpi3[['inf_3ma_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cpi3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['inf_3ma_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_inf3ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_inf3ma_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_rgdp_yoy']]

X = sm.add_constant(data_cpi4[['inf_3ma_dev', 'output_gap_rbb', 'opr_lag']])
model = sm.OLS(data_cpi4['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_3ma_dev']
coef_output_gap = coefficients['output_gap_rbb']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf3ma_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_rbb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_rbb'] / (1 - alpha)]
results_table_ols_inf3ma_rbb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_rbb']]


# GMM
exog = np.column_stack((np.ones_like(data_cpi1['opr_lag']), data_cpi1['inf_3ma_dev'], data_cpi1['output_gap_plucking'], data_cpi1['opr_lag']))
endog = data_cpi1['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi1['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cpi1['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi1['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cpi1['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_inf3ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi2['opr_lag']), data_cpi2['inf_3ma_dev'], data_cpi2['output_gap_bb_avg'], data_cpi2['opr_lag']))
endog = data_cpi2['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi2['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi2['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi2['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi2['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_inf3ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi3['opr_lag']), data_cpi3['inf_3ma_dev'], data_cpi3['output_gap_rgdp_yoy'], data_cpi3['opr_lag']))
endog = data_cpi3['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cpi3['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cpi3['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_inf3ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi4['opr_lag']), data_cpi4['inf_3ma_dev'], data_cpi4['output_gap_rbb'], data_cpi4['opr_lag']))
endog = data_cpi4['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi4['output_gap_rbb'], 1)[2:]
instruments_2 = np.roll(data_cpi4['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi4['output_gap_rbb'], 2)[2:]
instruments_5 = np.roll(data_cpi4['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rbb = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rbb / (1 - alpha)
results_table_gmm_inf3ma_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_rbb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_rbb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

In [5]:
# Sensitivity analysis using inf5ma

# OLS
X = sm.add_constant(data_cpi1[['inf_5ma_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cpi1['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_5ma_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf5ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_inf5ma_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_plucking']]

X = sm.add_constant(data_cpi2[['inf_5ma_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cpi2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_5ma_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf5ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_inf5ma_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_bb_avg']]

X = sm.add_constant(data_cpi3[['inf_5ma_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cpi3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['inf_5ma_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_inf5ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_inf5ma_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_rgdp_yoy']]

X = sm.add_constant(data_cpi4[['inf_5ma_dev', 'output_gap_rbb', 'opr_lag']])
model = sm.OLS(data_cpi4['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_5ma_dev']
coef_output_gap = coefficients['output_gap_rbb']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf5ma_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_rbb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_rbb'] / (1 - alpha)]
results_table_ols_inf5ma_rbb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_rbb']]

# GMM
exog = np.column_stack((np.ones_like(data_cpi1['opr_lag']), data_cpi1['inf_5ma_dev'], data_cpi1['output_gap_plucking'], data_cpi1['opr_lag']))
endog = data_cpi1['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi1['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cpi1['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi1['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cpi1['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_inf5ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi2['opr_lag']), data_cpi2['inf_5ma_dev'], data_cpi2['output_gap_bb_avg'], data_cpi2['opr_lag']))
endog = data_cpi2['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi2['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi2['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi2['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi2['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_inf5ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi3['opr_lag']), data_cpi3['inf_5ma_dev'], data_cpi3['output_gap_rgdp_yoy'], data_cpi3['opr_lag']))
endog = data_cpi3['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cpi3['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cpi3['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_inf5ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi4['opr_lag']), data_cpi4['inf_5ma_dev'], data_cpi4['output_gap_rbb'], data_cpi4['opr_lag']))
endog = data_cpi4['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi4['output_gap_rbb'], 1)[2:]
instruments_2 = np.roll(data_cpi4['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi4['output_gap_rbb'], 2)[2:]
instruments_5 = np.roll(data_cpi4['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rbb = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rbb / (1 - alpha)
results_table_gmm_inf5ma_rbb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_rbb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_rbb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_rbb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_rbb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

In [6]:
# Table 3, 5, 6
# Consolidate sensitivity analysis results

sensitivity_results_table = pd.concat([ results_table_ols_inf_plucking, results_table_gmm_inf_plucking, 
                                   results_table_ols_inf3ma_plucking, results_table_gmm_inf3ma_plucking, 
                                   results_table_ols_inf5ma_plucking, results_table_gmm_inf5ma_plucking, 
                                   results_table_ols_inf_bb, results_table_gmm_inf_bb, 
                                   results_table_ols_inf3ma_bb, results_table_gmm_inf3ma_bb, 
                                   results_table_ols_inf5ma_bb, results_table_gmm_inf5ma_bb,
                                   results_table_ols_inf_rbb, results_table_gmm_inf_rbb, 
                                   results_table_ols_inf3ma_rbb, results_table_gmm_inf3ma_rbb, 
                                   results_table_ols_inf5ma_rbb, results_table_gmm_inf5ma_rbb, 
                                   results_table_ols_inf_rgdp, results_table_gmm_inf_rgdp, 
                                   results_table_ols_inf3ma_rgdp, results_table_gmm_inf3ma_rgdp, 
                                   results_table_ols_inf5ma_rgdp, results_table_gmm_inf5ma_rgdp], 
                                  ignore_index=True)

# Create labels for the models
ols_inf_plucking_label = 'OLS (Inf - Plucking)'
gmm_inf_plucking_label = 'GMM (Inf - Plucking)'
ols_inf3ma_plucking_label = 'OLS (Inf3ma - Plucking)'
gmm_inf3ma_plucking_label = 'GMM (Inf3ma - Plucking)'
ols_inf5ma_plucking_label = 'OLS (Inf5ma - Plucking)'
gmm_inf5ma_plucking_label = 'GMM (Inf5ma - Plucking)'
ols_inf_bb_label = 'OLS (Inf - BB)'
gmm_inf_bb_label = 'GMM (Inf - BB)'
ols_inf3ma_bb_label = 'OLS (Inf3ma - BB)'
gmm_inf3ma_bb_label = 'GMM (Inf3ma - BB)'
ols_inf5ma_bb_label = 'OLS (Inf5ma - BB)'
gmm_inf5ma_bb_label = 'GMM (Inf5ma - BB)'
ols_inf_rbb_label = 'OLS (Inf - RBB)'
gmm_inf_rbb_label = 'GMM (Inf - RBB)'
ols_inf3ma_rbb_label = 'OLS (Inf3ma - RBB)'
gmm_inf3ma_rbb_label = 'GMM (Inf3ma - RBB)'
ols_inf5ma_rbb_label = 'OLS (Inf5ma - RBB)'
gmm_inf5ma_rbb_label = 'GMM (Inf5ma - RBB)'
ols_inf_rgdp_label = 'OLS (Inf - RGDP)'
gmm_inf_rgdp_label = 'GMM (Inf - RGDP)'
ols_inf3ma_rgdp_label = 'OLS (Inf3ma - RGDP)'
gmm_inf3ma_rgdp_label = 'GMM (Inf3ma - RGDP)'
ols_inf5ma_rgdp_label = 'OLS (Inf5ma - RGDP)'
gmm_inf5ma_rgdp_label = 'GMM (Inf5ma - RGDP)'

# Create a new column for model labels
model_labels = ([ols_inf_plucking_label] * len(results_table_ols_inf_plucking) + 
               [gmm_inf_plucking_label] * len(results_table_gmm_inf_plucking) + 
               [ols_inf3ma_plucking_label] * len(results_table_ols_inf3ma_plucking) + 
               [gmm_inf3ma_plucking_label] * len(results_table_gmm_inf3ma_plucking) + 
               [ols_inf5ma_plucking_label] * len(results_table_ols_inf5ma_plucking) + 
               [gmm_inf5ma_plucking_label] * len(results_table_gmm_inf5ma_plucking) + 
               [ols_inf_bb_label] * len(results_table_ols_inf_bb) + 
               [gmm_inf_bb_label] * len(results_table_gmm_inf_bb) + 
               [ols_inf3ma_bb_label] * len(results_table_ols_inf3ma_bb) + 
               [gmm_inf3ma_bb_label] * len(results_table_gmm_inf3ma_bb) + 
               [ols_inf5ma_bb_label] * len(results_table_ols_inf5ma_bb) + 
               [gmm_inf5ma_bb_label] * len(results_table_gmm_inf5ma_bb) +
               [ols_inf_rbb_label] * len(results_table_ols_inf_rbb) + 
               [gmm_inf_rbb_label] * len(results_table_gmm_inf_rbb) + 
               [ols_inf3ma_rbb_label] * len(results_table_ols_inf3ma_rbb) + 
               [gmm_inf3ma_rbb_label] * len(results_table_gmm_inf3ma_rbb) + 
               [ols_inf5ma_rbb_label] * len(results_table_ols_inf5ma_rbb) + 
               [gmm_inf5ma_rbb_label] * len(results_table_gmm_inf5ma_rbb) +  
               [ols_inf_rgdp_label] * len(results_table_ols_inf_rgdp) + 
               [gmm_inf_rgdp_label] * len(results_table_gmm_inf_rgdp) + 
               [ols_inf3ma_rgdp_label] * len(results_table_ols_inf3ma_rgdp) + 
               [gmm_inf3ma_rgdp_label] * len(results_table_gmm_inf3ma_rgdp) + 
               [ols_inf5ma_rgdp_label] * len(results_table_ols_inf5ma_rgdp) + 
               [gmm_inf5ma_rgdp_label] * len(results_table_gmm_inf5ma_rgdp))

# Add the new column to the merged results table
sensitivity_results_table.insert(0, 'Model', model_labels)

# Add a row to showcase the significance based on p-values
sensitivity_results_table['Significance'] = sensitivity_results_table['P-value'].apply(lambda p: '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else '')


# Round the values in the "Estimated Value" column to 3 significant figures
sensitivity_results_table["Estimated Value"] = sensitivity_results_table["Estimated Value"].round(3)
sensitivity_results_table["Standard Error"] = sensitivity_results_table["Standard Error"].round(3)

for i, row in sensitivity_results_table.iterrows():
    sensitivity_results_table.at[i, "Estimated Value"] = f"{row['Estimated Value']} {row['Significance']}"
    sensitivity_results_table.at[i, "Significance"] = ''

# Drop the "Significance" and "P-value" columns
sensitivity_results_table.drop(columns=["Significance", "P-value"], inplace=True)

# Iterate over each row of the sensitivity_results_table
for index, row in sensitivity_results_table.iterrows():
    # Get the Model name
    model = row["Model"]
    # Iterate over the columns and find the corresponding Standard Error
    for col in sensitivity_results_table.columns:
        if col.endswith("Standard Error"):
            standard_error_col = col
            break
    # Update the Estimated Value with Standard Error
    sensitivity_results_table.at[index, "Estimated Value"] += " (" + str(row[standard_error_col]) + ")"

# Drop the Standard Error columns
sensitivity_results_table.drop(columns=sensitivity_results_table.filter(like="Standard Error").columns, inplace=True)

# Pivot the table
sensitivity_results_table = sensitivity_results_table.pivot(index="Coefficient", columns="Model", values=["Estimated Value"])

sensitivity_results_table


Estimated Value                                          \
Model           GMM (Inf - BB) GMM (Inf - Plucking)    GMM (Inf - RBB)   
Coefficient                                                              
alpha        0.483 *** (0.146)    0.955 *** (0.044)  0.838 *** (0.048)   
beta         0.135 *** (0.047)      0.827 ** (0.35)   0.193 ** (0.075)   
gamma        0.178 *** (0.054)      -0.014  (0.073)   0.33 *** (0.091)   
r            2.805 *** (0.777)        3.01  (2.949)  2.999 *** (0.859)   

                                                                           \
Model         GMM (Inf - RGDP)  GMM (Inf3ma - BB) GMM (Inf3ma - Plucking)   
Coefficient                                                                 
alpha        0.949 *** (0.037)   0.754 *** (0.09)       0.931 *** (0.045)   
beta            0.113  (0.203)  -0.407 ** (0.176)          0.173  (0.691)   
gamma         0.568 *** (0.15)  0.231 *** (0.088)          0.011  (0.064)   
r               2.832  (2.195)  2.904 *** (1.031)         3.195 * (1.933)   

                                                                        \
Model       GMM (Inf3ma - RBB) GMM (Inf3ma - RGDP)   GMM (Inf5ma - BB)   
Coefficient                                                              
alpha         0.77 *** (0.048)   0.911 *** (0.052)   0.733 *** (0.101)   
beta           -0.049  (0.149)       0.52  (0.428)  -0.562 *** (0.164)   
gamma         0.349 *** (0.07)   0.334 *** (0.088)    0.196 ** (0.085)   
r            3.039 *** (0.619)     2.822 * (1.691)   2.953 *** (1.065)   

                                     ...                     \
Model       GMM (Inf5ma - Plucking)  ...    OLS (Inf - RBB)   
Coefficient                          ...                      
alpha              0.98 *** (0.034)  ...  0.835 *** (0.047)   
beta                  -4.252  (3.2)  ...     0.043  (0.082)   
gamma               -0.105  (0.244)  ...  0.274 *** (0.095)   
r                    3.787  (5.496)  ...  2.926 *** (0.836)   

                                                                           \
Model         OLS (Inf - RGDP)  OLS (Inf3ma - BB) OLS (Inf3ma - Plucking)   
Coefficient                                                                 
alpha        0.909 *** (0.037)  0.768 *** (0.059)       0.919 *** (0.063)   
beta            0.245  (0.202)    -0.085  (0.257)          0.304  (0.937)   
gamma           0.187  (0.143)  0.181 *** (0.066)         0.073 * (0.042)   
r             2.611 ** (1.171)   2.839 *** (0.73)          3.272  (2.299)   

                                                                       \
Model       OLS (Inf3ma - RBB) OLS (Inf3ma - RGDP)  OLS (Inf5ma - BB)   
Coefficient                                                             
alpha        0.836 *** (0.042)   0.904 *** (0.056)   0.78 *** (0.066)   
beta            0.016  (0.268)      0.255  (0.704)    -0.323  (0.264)   
gamma        0.284 *** (0.084)     0.221 * (0.132)   0.193 *** (0.06)   
r            2.938 *** (0.761)      2.604  (1.722)  2.882 *** (0.858)   

                                                                            
Model       OLS (Inf5ma - Plucking) OLS (Inf5ma - RBB) OLS (Inf5ma - RGDP)  
Coefficient                                                                 
alpha             0.952 *** (0.042)  0.835 *** (0.045)   0.929 *** (0.037)  
beta                 -0.61  (1.738)     0.038  (0.283)     -0.052  (1.083)  
gamma                0.089  (0.083)  0.284 *** (0.079)       0.29  (0.181)  
r                    3.536  (2.689)  2.933 *** (0.807)     2.595 * (1.525)  

[4 rows x 24 columns]

In [7]:
# Table 4
# Consolidate sensitivity analysis results

cinf_results_table = pd.concat([results_table_ols_cinf_plucking, results_table_gmm_cinf_plucking, 
                                   results_table_ols_cinf_bb, results_table_gmm_cinf_bb,  
                                   results_table_ols_cinf_rbb, results_table_gmm_cinf_rbb, 
                                   results_table_ols_cinf_rgdp, results_table_gmm_cinf_rgdp,], 
                                  ignore_index=True)

# Create labels for the models
ols_cinf_plucking_label = 'OLS (Core Inf - Plucking)'
gmm_cinf_plucking_label = 'GMM (Core Inf - Plucking)'
ols_cinf_bb_label = 'OLS (Core Inf - BB)'
gmm_cinf_bb_label = 'GMM (Core Inf - BB)'
ols_cinf_rbb_label = 'OLS (Core Inf - RBB)'
gmm_cinf_rbb_label = 'GMM (Core Inf - RBB)'
ols_cinf_rgdp_label = 'OLS (Core Inf - RGDP)'
gmm_cinf_rgdp_label = 'GMM (Core Inf - RGDP)'

# Create a new column for model labels
model_labels = ([ols_cinf_plucking_label] * len(results_table_ols_cinf_plucking) + 
               [gmm_cinf_plucking_label] * len(results_table_gmm_cinf_plucking) + 
               [ols_cinf_bb_label] * len(results_table_ols_cinf_bb) + 
               [gmm_cinf_bb_label] * len(results_table_gmm_cinf_bb) + 
               [ols_cinf_rbb_label] * len(results_table_ols_cinf_rbb) + 
               [gmm_cinf_rbb_label] * len(results_table_gmm_cinf_rbb) + 
               [ols_cinf_rgdp_label] * len(results_table_ols_cinf_rgdp) + 
               [gmm_cinf_rgdp_label] * len(results_table_gmm_cinf_rgdp))

# Add the new column to the merged results table
cinf_results_table.insert(0, 'Model', model_labels)

# Add a row to showcase the significance based on p-values
cinf_results_table['Significance'] = cinf_results_table['P-value'].apply(lambda p: '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else '')


# Round the values in the "Estimated Value" column to 3 significant figures
cinf_results_table["Estimated Value"] = cinf_results_table["Estimated Value"].round(3)
cinf_results_table["Standard Error"] = cinf_results_table["Standard Error"].round(3)

for i, row in cinf_results_table.iterrows():
    cinf_results_table.at[i, "Estimated Value"] = f"{row['Estimated Value']} {row['Significance']}"
    cinf_results_table.at[i, "Significance"] = ''

# Drop the "Significance" and "P-value" columns
cinf_results_table.drop(columns=["Significance", "P-value"], inplace=True)

# Iterate over each row of the cinf_results_table
for index, row in cinf_results_table.iterrows():
    # Get the Model name
    model = row["Model"]
    # Iterate over the columns and find the corresponding Standard Error
    for col in cinf_results_table.columns:
        if col.endswith("Standard Error"):
            standard_error_col = col
            break
    # Update the Estimated Value with Standard Error
    cinf_results_table.at[index, "Estimated Value"] += " (" + str(row[standard_error_col]) + ")"

# Drop the Standard Error columns
cinf_results_table.drop(columns=cinf_results_table.filter(like="Standard Error").columns, inplace=True)

# Pivot the table
cinf_results_table = cinf_results_table.pivot(index="Coefficient", columns="Model", values=["Estimated Value"])

cinf_results_table



Estimated Value                            \
Model       GMM (Core Inf - BB) GMM (Core Inf - Plucking)   
Coefficient                                                 
alpha           0.392 ** (0.19)         0.948 *** (0.038)   
beta             0.069  (0.051)            0.602  (0.464)   
gamma         0.212 *** (0.067)            -0.001  (0.07)   
r             2.811 *** (0.863)            3.178  (2.204)   

                                                                            \
Model       GMM (Core Inf - RBB) GMM (Core Inf - RGDP) OLS (Core Inf - BB)   
Coefficient                                                                  
alpha          0.777 *** (0.056)     0.946 *** (0.046)   0.746 *** (0.076)   
beta               0.03  (0.078)        0.525  (0.405)      0.036  (0.103)   
gamma          0.368 *** (0.083)     0.484 *** (0.145)   0.163 *** (0.054)   
r              3.018 *** (0.756)        2.925  (2.562)   2.823 *** (0.843)   

                                                            \
Model       OLS (Core Inf - Plucking) OLS (Core Inf - RBB)   
Coefficient                                                  
alpha               0.942 *** (0.042)    0.846 *** (0.043)   
beta                  -0.025  (0.662)      -0.088  (0.156)   
gamma                  0.094  (0.068)    0.306 *** (0.087)   
r                      3.459  (2.234)     2.96 *** (0.835)   

                                   
Model       OLS (Core Inf - RGDP)  
Coefficient                        
alpha            0.933 *** (0.04)  
beta              -0.147  (0.551)  
gamma              0.319  (0.201)  
r                  2.581  (1.884)